# 1. Cuestionario del alumno

In [104]:
from scipy import spatial
import numpy as np
import math
from itertools import islice

def take(n, iterable):
    """Return the first n items of the iterable as a list."""
    return list(islice(iterable, n))

In [105]:
BD = "proyecto_final"

In [106]:
dic_campos_bd = {
     1: "arte_diseno_arquitectura",
     2: "agricultura_forestal",
     3: "empresariales_administracion",
     4: "economicas",
     5: "educacion",
     6: "ingenieria_tecnologia",
     7: "historia_filosofia_teologia",
     8: "leyes_abogacia",
     9: "ciencias_salud",
    10: "ciencias_naturales",
    11: "ciencias_sociales",
    12: "otros",
}

dic_campos_usuario = {
    "arte_diseno_arquitectura": "Arte, diseño y arquitectura",
    "agricultura_forestal": "Agricultura y forestal",
    "empresariales_administracion": "Empresariales y administración",
    "economicas": "Economía",
    "educacion": "Educación",
    "ingenieria_tecnologia": "Ingeniería y tecnología",
    "historia_filosofia_teologia": "Historia, filosofía y teología",
    "leyes_abogacia": "Leyes y abogacía",
    "ciencias_salud": "Ciencias de la salud",
    "ciencias_naturales": "Ciencias naturales",
    "ciencias_sociales": "Ciencias sociales",
    "otros": "Otros",
}

In [131]:
print("Introduce el identificador de 3 campos de tu interés de mayor a menor preferencia:")
for key, value in dic_campos_bd.items():
    print(f"{'{:3d}'.format(key)}. {dic_campos_usuario[value]}")
print()

completed = False
selected_ids = []
max_length = 3
while not completed:
    selection = input(f"Introduce el identificador de tu elección ({len(selected_ids)}/{max_length}): ")
    if selection not in selected_ids:
        print(f"    Has elegido {dic_campos_usuario[dic_campos_bd[int(selection)]]}")
        selected_ids.append(selection)
        if len(selected_ids) == max_length:
            completed = True
    else:
        print("    Error: Ya has escogido ese campo")

Introduce el identificador de 3 campos de tu interés de mayor a menor preferencia:
  1. Arte, diseño y arquitectura
  2. Agricultura y forestal
  3. Empresariales y administración
  4. Economía
  5. Educación
  6. Ingeniería y tecnología
  7. Historia, filosofía y teología
  8. Leyes y abogacía
  9. Ciencias de la salud
 10. Ciencias naturales
 11. Ciencias sociales
 12. Otros

Introduce el identificador de tu elección (0/3): 1
    Has elegido Arte, diseño y arquitectura
Introduce el identificador de tu elección (1/3): 7
    Has elegido Historia, filosofía y teología
Introduce el identificador de tu elección (2/3): 8
    Has elegido Leyes y abogacía


In [132]:
proj_usuario = [10, 5, 2, 1, 1, 1]
print(selected_ids)

['1', '7', '8']


Sentencia para obtener el ranking de las universidades en los campos elegidos por el usuario

In [133]:
sentencia_select = f"SELECT {BD}.universidades.nombre AS nombre_univ, {BD}.campos.nombre AS nombre_campo, {BD}.rankings.id_univ, {BD}.rankings.id_campo, {BD}.rankings.puntuacion FROM {BD}.campos"
sentencia_join = f"""INNER JOIN {BD}.rankings ON {BD}.campos.id = {BD}.rankings.id_campo
                     INNER JOIN {BD}.universidades ON {BD}.universidades.id = {BD}.rankings.id_univ"""
ids_str = ','.join(selected_ids)
sentencia_where = f"WHERE {BD}.campos.id IN ({','.join(selected_ids)})"

sentencia_sql = f"""{sentencia_select}
                    {sentencia_join}
                    {sentencia_where}"""


rankings_df = spark.sql(sentencia_sql).fillna(0)
rankings_df.show()

+--------------------+--------------------+-------+--------+----------+
|         nombre_univ|        nombre_campo|id_univ|id_campo|puntuacion|
+--------------------+--------------------+-------+--------+----------+
|Universidad Nacio...|arte_diseno_arqui...|     36|       1|         1|
|Universidad Nacio...|historia_filosofi...|     36|       7|         4|
|Universidad Nacio...|      leyes_abogacia|     36|       8|         5|
|Universidad Obert...|arte_diseno_arqui...|     37|       1|         5|
|Universidad Obert...|historia_filosofi...|     37|       7|         1|
|Universidad Obert...|      leyes_abogacia|     37|       8|         5|
|Universidad Polit...|arte_diseno_arqui...|     41|       1|         7|
|Universidad Polit...|historia_filosofi...|     41|       7|         1|
|Universidad Polit...|      leyes_abogacia|     41|       8|         0|
|Universidad de Ba...|arte_diseno_arqui...|     57|       1|         6|
|Universidad de Ba...|historia_filosofi...|     57|       7|    

In [134]:
list_of_unis = zip(*(iter(rankings_df.collect()),) * 3)

In [135]:
unis_dict = {}
for campo_1, campo_2, campo_3 in list_of_unis:
    uni_dict = {campo_1['id_campo']: campo_1['puntuacion'],
                campo_2['id_campo']: campo_2['puntuacion'],
                campo_3['id_campo']: campo_3['puntuacion'],
                'especialidad1': -1,
                'especialidad2': -1,
                'especialidad3': -1
                }
    unis_dict[campo_1['id_univ']] = uni_dict

In [136]:
def calcular_especialidad(unis_dict, id_especialidad, especialidad_key):
    sentencia_select = f"SELECT {BD}.universidades.id AS id_univ, {BD}.universidades.nombre AS nombre_univ, {BD}.campos.nombre AS nombre_campo, COUNT(*) AS carreras FROM {BD}.campos"
    sentencia_join = f"""INNER JOIN {BD}.facultades ON {BD}.campos.id = {BD}.facultades.id_campo INNER JOIN {BD}.grados ON {BD}.grados.id_fac = {BD}.facultades.id INNER JOIN {BD}.universidades ON {BD}.universidades.id = {BD}.facultades.id_univ"""

    ids_str = ','.join(selected_ids)
    sentencia_where = f"WHERE {BD}.campos.id IN ({id_especialidad})"
    sentencia_group_by = f"GROUP BY {BD}.campos.nombre, {BD}.universidades.nombre, {BD}.universidades.id"
    sentencia_sql = f"""{sentencia_select} {sentencia_join} {sentencia_where} {sentencia_group_by} ORDER BY COUNT(*) DESC"""
    
    carreras_total_df = spark.sql(sentencia_sql).collect()
    max_especialidad = carreras_total_df[0]['carreras']
    for row in carreras_total_df:
        id_univ = row['id_univ']
        num_carreras = row['carreras']
        unis_dict[id_univ][especialidad_key] = math.log(num_carreras, max_especialidad)

In [137]:
calcular_especialidad(unis_dict, selected_ids[0], 'especialidad1')
calcular_especialidad(unis_dict, selected_ids[1], 'especialidad2')
calcular_especialidad(unis_dict, selected_ids[2], 'especialidad3')
print(unis_dict)

{36: {1: 1, 7: 4, 8: 5, 'especialidad1': -1, 'especialidad2': 0.4484876484351508, 'especialidad3': 0.35436764027111584}, 37: {1: 5, 7: 1, 8: 5, 'especialidad1': -1, 'especialidad2': -1, 'especialidad3': -1}, 41: {1: 7, 7: 1, 8: 0, 'especialidad1': 0.7586544129592989, 'especialidad2': -1, 'especialidad3': -1}, 57: {1: 6, 7: 12, 8: 16, 'especialidad1': 0.42831734103139474, 'especialidad2': 0.6295320036582305, 'especialidad3': 0.45801353078493934}, 66: {1: 3, 7: 12, 8: 15, 'especialidad1': 0.42831734103139474, 'especialidad2': 0.8537758278758059, 'especialidad3': 0.45801353078493934}, 74: {1: 0, 7: 3, 8: 11, 'especialidad1': 0.27023815442731974, 'especialidad2': 0.6727314726527263, 'especialidad3': 0.17718382013555792}, 77: {1: 4, 7: 19, 8: 18, 'especialidad1': 0.0, 'especialidad2': 0.5796618766207391, 'especialidad3': 0.35436764027111584}, 87: {1: 0, 7: 2, 8: 6, 'especialidad1': -1, 'especialidad2': -1, 'especialidad3': -1}, 8: {1: 0, 7: 0, 8: 0, 'especialidad1': 0.42831734103139474, 'es

In [164]:
def calculate_uni_projection(uni_dict, selected_ids):
    """
    """
    total_investigators = 0
    for investigators in uni_dict.values():
        total_investigators += investigators
    
    if total_investigators == 0:
        return [-100, -100, -100, -1, -1, -1]
    # For each field, ordered by user preference, calculate the percentage of total investigators
    
    kv_pairs = take(3, uni_dict.items())
    sub_dict = {}
    for k, v in kv_pairs:
        sub_dict[k] = v
    ordered_dict = sorted(sub_dict.items(), key=lambda x:x[1], reverse=True)
    punctuations_dict = {}
    
    
    if ordered_dict[0][1] == 0:
        print(sub_dict)
        punctuations_dict[ordered_dict[0][0]] = -100
    else:
        punctuations_dict[ordered_dict[0][0]] = 10
    
    if ordered_dict[1][1] == 0:
        punctuations_dict[ordered_dict[1][0]] = -100
    else:
        punctuations_dict[ordered_dict[1][0]] = 5
    
    if ordered_dict[2][1] == 0:
        punctuations_dict[ordered_dict[2][0]] = -100
    else:
        punctuations_dict[ordered_dict[2][0]] = 2
    
    uni_proj = []
    for campo in selected_ids:
        percentage = uni_dict[int(campo)]
        uni_proj.append(punctuations_dict[int(campo)])
    uni_proj.append(uni_dict['especialidad1'])
    uni_proj.append(uni_dict['especialidad2'])
    uni_proj.append(uni_dict['especialidad3']) 
        
    return uni_proj

In [168]:
projs_dict[8]

[-100, -100, -100, 0.42831734103139474, -1, -1]

In [167]:
projs_dict = {}
for key, value in unis_dict.items():
    print(key)
    projs_dict[key] = calculate_uni_projection(value, selected_ids)
print(projs_dict)

36
37
41
57
66
74
77
87
8
{1: 0, 7: 0, 8: 0}
10
11
12
13
14
16
{1: 0, 7: 0, 8: 0}
51
44
18
2
{1: 0, 7: 0, 8: 0}
3
22
24
21
{1: 0, 7: 0, 8: 0}
23
{1: 0, 7: 0, 8: 0}
26
5
{1: 0, 7: 0, 8: 0}
30
32
29
{1: 0, 7: 0, 8: 0}
33
48
34
20
{1: 0, 7: 0, 8: 0}
35
{1: 0, 7: 0, 8: 0}
9
52
38
39
42
43
45
46
47
49
17
15
50
40
28
53
54
55
{1: 0, 7: 0, 8: 0}
56
{1: 0, 7: 0, 8: 0}
58
61
59
60
62
63
64
65
67
68
69
84
70
71
72
7
{1: 0, 7: 0, 8: 0}
73
75
76
78
79
85
86
88
80
81
82
83
4
1
27
{1: 0, 7: 0, 8: 0}
25
6
31
{1: 0, 7: 0, 8: 0}
19
{1: 0, 7: 0, 8: 0}
{36: [2, 5, 10, -1, 0.4484876484351508, 0.35436764027111584], 37: [10, 2, 5, -1, -1, -1], 41: [10, 5, -100, 0.7586544129592989, -1, -1], 57: [2, 5, 10, 0.42831734103139474, 0.6295320036582305, 0.45801353078493934], 66: [2, 5, 10, 0.42831734103139474, 0.8537758278758059, 0.45801353078493934], 74: [-100, 5, 10, 0.27023815442731974, 0.6727314726527263, 0.17718382013555792], 77: [2, 10, 5, 0.0, 0.5796618766207391, 0.35436764027111584], 87: [-100, 5, 10, -1, -1

In [170]:
def get_three_most_similar_unis(projs_dict, proj_user):
    """
        Searches for the three most similar unis to the user preferences
    """
    np_user_proj = np.array(proj_user)
    similarities_dict = {}
    for uni_key, proj in projs_dict.items():
        np_uni_proj = np.array(proj)
        similarities_dict[uni_key] = np.linalg.norm(np_user_proj-np_uni_proj)
        
    similarities_ordered = sorted(similarities_dict.items(), key=lambda x:x[1])
    return similarities_ordered[:3]

In [171]:
recomendations = get_three_most_similar_unis(projs_dict, proj_usuario)
print(recomendations)

[(37, 5.477225575051661), (80, 7.163634109546113), (69, 7.416457882871859)]


In [172]:
print(sentencia_sql)

SELECT proyecto_final.universidades.nombre AS nombre_univ, proyecto_final.campos.nombre AS nombre_campo, proyecto_final.rankings.id_univ, proyecto_final.rankings.id_campo, proyecto_final.rankings.puntuacion FROM proyecto_final.campos
                    INNER JOIN proyecto_final.rankings ON proyecto_final.campos.id = proyecto_final.rankings.id_campo
                     INNER JOIN proyecto_final.universidades ON proyecto_final.universidades.id = proyecto_final.rankings.id_univ
                    WHERE proyecto_final.campos.id IN (1,7,8)


In [173]:

print(sentencia_sql)
SELECT universidades.nombre AS nombre_univ, campos.nombre AS nombre_campo, COUNT(*) AS carreras FROM campos INNER JOIN facultades ON campos.id = facultades.id_campo INNER JOIN grados ON grados.id_fac = facultades.id INNER JOIN universidades ON universidades.id = facultades.id_univ WHERE campos.id IN (4,5,11) GROUP BY campos.nombre, universidades.nombre

SyntaxError: invalid syntax (3252934731.py, line 2)

In [174]:
SELECT universidades.nombre AS nombre_univ, campos.nombre AS nombre_campo, rankings.id_univ, rankings.id_campo, rankings.puntuacion FROM campos INNER JOIN rankings ON campos.id = rankings.id_campo INNER JOIN universidades ON universidades.id = rankings.id_univ WHERE campos.id IN (4,5,11)

SyntaxError: invalid syntax (198619458.py, line 1)